In [1]:
import datetime
import numpy as np
from copy import deepcopy

In [2]:
class State:

    def __init__(self, state, pre_move = 'root'):
        self.state = np.array([state[:3], state[3:6], state[6:]])
        self.pre_move = pre_move
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, State):
            return np.array_equal(self.state, other.state)
        
    def __hash__(self):
        return hash(tuple(map(tuple, self.state)))

    def FindZero(self):
        index = np.where(self.state == 0)
        coor = list(zip(index[0], index[1]))
        return list(coor[0])
    
    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes

In [3]:
visited = {}
path = []
n_expand = 0

def DFS(state, goal, depth, limit):
    global visited, path, n_expand
    if state in visited:
        return False, False
    visited[state] = True
    if depth == 0:
        if state == goal:
            path.append(state.pre_move)
            return True, False
    else:
        if n_expand >= limit:
            return False, True
        for node in state.Expand():
            n_expand +=1
            found, maxed = DFS(node, goal, depth-1, limit)
            if found:
                path.append(state.pre_move)
                return True, False
            if maxed:
                return False, True
    del visited[state]
    return False, False

def ItertiveDFS(start, goal, limit):
    global n_expand
    depth = 0
    while True:
        n_expand = 0
        found, maxed = DFS(start, goal, depth, limit)

        if found:
            path.reverse()
            print("Path found:", path)
            print('Number of moves:', len(path)-1)
            print('Number of nodes expaned: ', n_expand)
            break
        if maxed:
            print('Limit reached')
            print('Number of nodes expaned: ', n_expand)
            break
        depth += 1
    print('Depth used =', depth)
    return

In [4]:
if __name__ == '__main__':
    start = [7,2,4,5,0,6,8,3,1]
    goal = [0,1,2,3,4,5,6,7,8]
    start_state = State(start)
    goal_state = State(goal)
    a = datetime.datetime.now()
    ItertiveDFS(start_state, goal_state, 1000000)
    b = datetime.datetime.now()
    print("Time used: ", b - a)
    #Run time is about 3 mintues

Limit reached
Number of nodes expaned:  1000000
Depth used = 23
Time used:  0:02:38.073137
